
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
#D'abord télécharger les données en allant dans create table et en chargent les données  dans databricks
#On copie les données de la tables afin d'y avoir accés , les voici
# File location and type
file_location = "/FileStore/tables/mushrooms.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#Si on veut voir nos données ,on peut utiliser
# display(df)

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "mushrooms_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

#On encode les valeurs

# Supposons que "class" est la variable cible
label = "class"

# Créer un StringIndexer pour chaque colonne catégorielle
categorical_cols = [col_name for col_name, data_type in df.dtypes if data_type == "string"]
indexers = [StringIndexer(inputCol=col_name, outputCol=col_name+"_index").fit(df) for col_name in categorical_cols]

# Créer un assembleur de vecteur pour combiner les caractéristiques en une seule colonne
feature_cols = [col_name+"_index" for col_name in categorical_cols]
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Créer un StringIndexer pour la variable cible "class"
label_indexer = StringIndexer(inputCol=label, outputCol="label")

# Créer un pipeline pour préparer les données
pipeline = Pipeline(stages=indexers + [vector_assembler, label_indexer])

# Adapter le pipeline aux données
model = pipeline.fit(df)
df_transformed = model.transform(df)

# Diviser les données en ensembles d'entraînement et de test
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)





In [0]:

#Voici mon premier modèle random forest 
# Modèle Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, seed=42)
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)

In [0]:
#Voici mon second modèle GBTClassifier
# Modèle GBTClassifier
xgb = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10, seed=42)
xgb_model = xgb.fit(train_data)
xgb_predictions = xgb_model.transform(test_data)


In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Évaluation des modèles
evaluator = BinaryClassificationEvaluator(labelCol="label")

rf_auc = evaluator.evaluate(rf_predictions)
xgb_auc = evaluator.evaluate(xgb_predictions)
print(f"Aire sous la courbe ROC pour Random Forest: {rf_auc}")
print(f"Aire sous la courbe ROC pour XGBoost: {xgb_auc}")


# Créez un évaluateur pour la précision
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

# Évaluez le modèle Random Forest
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Précision pour Random Forest: {rf_accuracy}")

# Évaluez le modèle XGBoost
xgb_accuracy = evaluator.evaluate(xgb_predictions)
print(f"Précision pour XGBoost: {xgb_accuracy}")


Aire sous la courbe ROC pour Random Forest: 1.0
Aire sous la courbe ROC pour XGBoost: 1.0
Précision pour Random Forest: 1.0
Précision pour XGBoost: 1.0
